#### **This script gets the list of Most Actives ticker symbols from the https://money.cnn.com/data/hotstocks/ website and data related to symbols such as open price, volume and pe ratio from https://www.google.com/finance/quote/Symbol:NYSE website. It also stores the data in stocks.txt(Text file) and StocksDatabase.db(Database file) file.**

#### Details of data which is collected from both websites:
1. Ticker Symb: It is a text value of most actives ticker symbols. For example: F represents symbol for Ford Motor Co.
2. OpenPrice: It is a value of open price of symbol. For example: DAY RANGE value of F is $18.61-$19.49. So, open price of F will be $18.61.
3. Volume: It is the value of volume of symbol. For example: Volume of F is 107.59M which is equivalent to 107590000.
4. P/E Ratio: It is the value of pe ratio of symbol. For example: P/E Ratio of F is 27.27.

In [1]:
#Imported Libraries and Modules
from bs4 import BeautifulSoup   #Used BeautifulSoup module for web scrapping
import requests                 #Used requests module for sending HTTP requests and to get response data
import re                       #Used re module for using regular expressions
import csv                      #Used csv module for generating file
import sqlite3                  #Used sqlite3 module for connecting to a SQLite database



symbol_doc=requests.get('https://money.cnn.com/data/hotstocks/') 
print("status code of https://money.cnn.com/data/hotstocks/ is:", symbol_doc.status_code)  #Get status code of the tickers url
soup=BeautifulSoup(symbol_doc.text, 'html.parser')  #Parsing HTML page
ma_tab=soup.find('table', class_='wsod_dataTable wsod_dataTableBigAlt')
ma_dic={}  #Initialized a dictionary
for i in ma_tab.find_all('a', class_='wsod_symbol'):  #Finding Most Active ticker symbol in ma_tab  
    st=i.text
    url_1='https://money.cnn.com/' + i.get('href')  #This will give the url with proper formatted symbol 
    ma_dic[st]=url_1                                #Storing symbols with url in dictionary

#This format "https://www.google.com/finance/quote/F" (Where F is Ford Motor Co) is not accepted by Google Finance
#Google Finance accepts format "https://www.google.com/finance/quote/F:NYSE"
for sy, url_ma in ma_dic.items():
    soup=BeautifulSoup(requests.get(url_ma).text, 'html.parser')  #Parsing HTML page               
    tic_txt=soup.find('span', class_='wsod_smallSubHeading')    
    j=':'.join(re.sub(r'[\(\)]', '', tic_txt.text).split(":")[::-1]) #It will get the proper format (For example: F:NYSE) and store in a variable
    ma_dic[sy]='https://www.google.com/finance/quote/' + j  #Storing google finance urls of symbols in dictionary

stocks_dic={} #Initialized a dictionary

for syl, url in ma_dic.items():
    soup_1=BeautifulSoup(requests.get(url).text, 'html.parser')   #Parsing HTML page
    gf_data=soup_1.find_all('div', class_='P6K39c')
    openPrice=float(gf_data[1].text.split(' - ')[0].replace('$', ''))    #Retrieving open price from HTML page in float type
    vol=float(float(gf_data[4].text.replace('M', '').replace(',', '')))  # Retrieving volume from HTML page in float type
    #Converting volume to integer type by multiplying it by 1000000 (For Example volume of F is 107.59M=107590000)
    volume=int(vol*1000000)  
    
    #Whenever the pe ratio is not there on webpage then it shows '-'. To counter this situation:
    if gf_data[5].text!='-':  #If pe ratio is present then retrieve its value in float type
        peRatio=float(gf_data[5].text)
    else:
        peRatio="NULL"        #If pe ratio is not present then show "NULL"
    stocks_dic[syl]={"open_price": openPrice,"volume": volume,"pe_ratio": peRatio} #Storing open price, volume and pe ratio in dictionary



stocks_file=open('stocks.txt', 'w')     #Create stocks.txt file
print("Text file stocks.txt is successfully created")
csv_writer=csv.writer(stocks_file)      #Used writer function of csv module to insert data in stocks.text file
csv_writer.writerow(['TickerSymb', 'OpenPrice', 'Volume', 'PERatio']) #Used writerow function of csv module to insert data in row format in stocks.text file

co=sqlite3.connect('StocksDatabase.db')   #Created StocksDatabase.db successfully
cu=co.cursor()  #Created cursor object for executing the query and to fetch the record from the database
print("Database is successfully created and connected to Sqlite")   


#Created StocksTable table in StocksDatabase.db database
cu.execute('''CREATE TABLE IF NOT EXISTS StocksTable
               (TickerSymb TEXT, OpenPrice REAL, Volume INTEGER, PERatio REAL)''')

#Insert data in stocks.text file and StocksDatabase.db file
for sm, data in stocks_dic.items():
    csv_writer.writerow([str(sm), data['open_price'], data['volume'], data['pe_ratio']])
    cu.execute('''
        INSERT INTO StocksTable VALUES ('{}', {}, {}, {})
    '''.format(sm, data['open_price'], data['volume'], data['pe_ratio'])
    )   
    
print("Data is successfully inserted in stocks.txt and StocksDatabase.db")

#Display StocksDatabase.db file records
cu.execute("SELECT * FROM StocksTable")   #Used execute function to execute query
res=cu.fetchall()  #Used fetchall function to retrieve the records from database file
print("StocksDatabase.db Database records are: ", res)

 
co.commit() #commit function of sqlite3 module is used to confirm the changes made to the database
co.close()  #close function sqlite3 module is used to close the database connection 
stocks_file.close()  #close function of csv module is used to close the file.
print("Sqlite database connection is closed")


status code of https://money.cnn.com/data/hotstocks/ is: 200
Text file stocks.txt is successfully created
Database is successfully created and connected to Sqlite
Data is successfully inserted in stocks.txt and StocksDatabase.db
StocksDatabase.db Database records are:  [('F', 15.57, 65, 5.28), ('BAC', 34.48, 35, 10.42), ('CCL', 9.71, 50, -1.0), ('T', 17.61, 35, 6.34), ('OXY', 70.9, 25, 6.76), ('VZ', 42.21, 20, 8.29), ('NCLH', 13.52, 18, -1.0), ('XOM', 96.79, 18, 10.44), ('HPE', 13.62, 10, 4.73), ('C', 50.15, 14, 6.24), ('F', 15.06, 65239999, 5.28), ('BAC', 33.27, 35300000, 10.42), ('CCL', 9.27, 50620000, None), ('T', 17.16, 35670000, 6.34), ('OXY', 68.27, 25070000, 6.76), ('VZ', 41.15, 20380000, 8.29), ('NCLH', 12.86, 18880000, None), ('XOM', 95.11, 18580000, 10.44), ('HPE', 13.18, 10500000, 4.73), ('C', 48.62, 14920000, 6.24)]
Sqlite database connection is closed
